#### ~80% of CVEs were found to be code related

In [1]:
# Filter distinct CVEs for known_affected extractd data
import pandas as pd

# Load the Excel file
file_path = 'known_not_affected.xlsx'  # Replace with the path to your Excel file
sheet_name = 'sheet1'  # Replace with the name of your sheet

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Extract distinct CVEs
distinct_cves = df['CVE'].drop_duplicates()

# Number of distinct CVEs
print(len(distinct_cves))
cves = distinct_cves.tolist()
# print(cves)

319


In [2]:


import os
import time
import json
import requests
import re
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
from typing import List, Dict

# Load environment variables
load_dotenv()

# Constants
NVD_API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"
NVD_API_KEY = os.getenv("NVD_API_KEY")
RATE_LIMIT_WAIT_TIME = 1
MAX_RETRIES = 3
MAX_CVE_PER_BATCH = 3  # Limit number of CVEs per batch to manage token count
MAX_DESCRIPTION_LENGTH = 200  # Limit length of descriptions
MAX_PACKAGES_SHOWN = 3  # Limit number of affected packages shown

# Enhanced LangChain configuration with improved prompt
template = """
You are an expert cybersecurity analyst specializing in vulnerability assessment. Your task is to analyze a list of CVE (Common Vulnerabilities and Exposures) identifiers and identify which ones are specifically code-related.

## What Makes a CVE Code-Related
A code-related CVE involves vulnerabilities directly attributable to software implementation issues rather than configuration, physical security, or policy problems. These include:

1. Remote Code Execution (RCE)
2. Code/Command Injection
3. SQL Injection
4. Cross-Site Scripting (XSS)
5. Deserialization vulnerabilities
6. Buffer Overflows
7. Format String vulnerabilities
8. Memory corruption issues
9. Path traversal that could lead to code execution
10. Arbitrary file inclusion
11. Logic flaws in application code
12. Implementation flaws in libraries, frameworks, or programming languages
13. Race conditions affecting code execution
14. Improper cryptographic implementations
15. Client-side/server-side request forgery
16. Coding errors leading to memory corruption (buffer/heap overflows, use-after-free)
17. Input validation failures (SQL injection, XSS, command injection, XXE)
18. Insecure API implementations or calls
19. Authentication/authorization bypass due to code flaws
20. Use-after-free vulnerabilities - When memory is accessed after being freed, potentially allowing attackers to execute arbitrary code
21. Integer overflow/underflow - When arithmetic operations exceed the maximum or minimum size of the integer type
22. Double-free vulnerabilities - When memory is freed twice, potentially corrupting memory allocation data structures
23. Null pointer dereference - When a program attempts to read or write to memory with a null pointer
24. Uninitialized memory usage - Using memory that hasn't been properly initialized
25. Out-of-bounds read/write - Accessing memory locations beyond array boundaries
26. DOM-based XSS - When client-side JavaScript modifies the DOM in an unsafe way
27. Prototype pollution - Manipulating JavaScript object prototype chains
28. XML External Entity (XXE) processing - When XML parsers process external entity references
29. Cross-Site Request Forgery (CSRF) - Tricking users into performing unwanted actions
30. HTTP request smuggling - Exploiting differences in how servers and proxies interpret HTTP requests
31. Insecure CORS configurations in code - Cross-Origin Resource Sharing misconfigurations
32. GraphQL query depth/complexity issues - When recursive queries can cause DoS conditions
33. OAuth 2.0 implementation flaws - Improper token validation or handling
34. JWT token validation failures - Missing signature verification or allowing algorithm switching
35. Broken function-level authorization - When API endpoints don't properly check permissions
36. API race conditions - When parallel requests can manipulate shared resources
37. Side-channel attacks in crypto implementations - Timing attacks, power analysis
38. Weak random number generation - Using predictable values for security-critical operations
39. Hard-coded cryptographic keys - Embedding secret keys in application code
40. Padding oracle vulnerabilities - When error messages reveal information about encryption
41. CBC mode implementation flaws - Improper initialization vectors or padding
42. Insecure deeplink handling - When mobile apps process deeplinks without proper validation
43. Insufficient binary protections - Missing ASLR, DEP, or code signing
44. Insecure WebView implementations - JavaScript bridge vulnerabilities
45. Insecure local storage of sensitive data - Storing credentials in plaintext
46. Certificate pinning bypass vulnerabilities - When TLS certificate validation can be circumvented
47. Unsafe deserialization in Jackson - Type confusion issues leading to RCE
48. Spring framework injection vulnerabilities - SpEL expression language injection
49. React XSS through dangerouslySetInnerHTML - Bypassing React's automatic escaping
50. Django template injection - Server-side template injection in Django applications
51. Log4Shell-type vulnerabilities - JNDI injection through logging frameworks
52. PHP object injection - Exploiting unserialize() on user-controlled data
53. Python pickle deserialization - Arbitrary code execution through pickle.loads()
54. Ruby on Rails mass assignment - When user input can set protected attributes
55. TypeScript/JavaScript prototype pollution - Manipulating object prototypes
56. Go concurrency bugs - Race conditions in goroutines
57. Container escape vulnerabilities - Breaking out of container isolation
58. VM escape vulnerabilities - Exploiting hypervisor bugs to access the host
59. Kubernetes API server vulnerabilities - Authentication or authorization bypass
60. Improper certificate validation - Not checking certificate validity or allowing self-signed certificates
61. Time-of-check to time-of-use (TOCTOU) race conditions - When a resource changes between checking and using it
62. Unsafe reflection usage - Dynamic code execution based on user input
63. Dependency confusion attacks - When package managers resolve dependencies incorrectly
64. Path normalization issues - Directory traversal through path manipulation
65. Logic bugs in authentication flows - Password reset or MFA implementation flaws





## For Each CVE, Determine:
1. Is it code-related? (yes/no)
2. If not code-related, specify the exclusion reason

## CVE Details to Analyze:
{cve_details}

## Response Format:
Return a JSON array of objects with the following format:
{{
  "cve_id": "The CVE identifier",
  "code_related": true/false,
  "exclusion_reason": "If not code-related, specify the reason from the exclusion criteria list below. Leave blank if code-related."
}}

## Exclusion Criteria (if not code-related):
1. Configuration errors only (e.g., misconfigured access controls, incorrect firewall rules, improper permission settings, insecure registry keys, misconfigured HTTP headers)
2. Physical security issues (e.g., unauthorized physical access, hardware tampering, cable disconnection, side-channel attacks, electromagnetic interference, power analysis attacks)
3. Social engineering without code exploits (e.g., phishing, pretexting, baiting, tailgating, shoulder surfing, dumpster diving, voice phishing/vishing)
4. Default credentials (e.g., unchanged factory passwords, standard admin accounts, hardcoded test accounts, vendor default usernames/passwords, unchanged SSH keys)
5. Policy/procedure violations (e.g., improper access management, missing governance controls, inadequate backup procedures, improper user offboarding, lack of least privilege enforcement)
6. Expired certificates (e.g., SSL/TLS certificate expiration, outdated cryptographic signatures, revoked certificates still in use, expired code signing certificates)
7. Denial of Service that doesn't involve code manipulation (e.g., resource exhaustion through legitimate requests, TCP/SYN flooding, ICMP flooding, bandwidth consumption attacks)
8. Hardware-based issues with no software component (e.g., power supply failures, CPU vulnerabilities like Spectre/Meltdown, faulty memory modules, compromised firmware that isn't patchable via software)
9. Documentation errors or insufficient warnings (e.g., misleading security guidance, incomplete installation instructions, undocumented features)
10. Missing security headers without exploitable code (e.g., absent Content-Security-Policy, missing X-Frame-Options, lack of HSTS)
11. Missing patches where the vulnerability is in the unpatched system (e.g., known OS vulnerabilities that simply need vendor updates applied)
12. Overly permissive CORS policies without actual code vulnerabilities (e.g., allowing all origins in CORS headers)
13. Information disclosure via error messages without exploitable code issues (e.g., stack traces exposed to users, SQL error details shown publicly)
14. Weak password policies without implementation flaws (e.g., allowing short passwords, not requiring complexity, no account lockout mechanisms)
15. Outdated libraries/dependencies without specific code-related exploitation (e.g., using deprecated frameworks without exploiting known CVEs)
16. Self-XSS requiring user interaction without system code flaws (e.g., requiring users to paste malicious code into their own browsers)
17. Missing rate limiting without associated code vulnerabilities (e.g., unlimited login attempts, excessive API requests)
18. Encrypted data storage issues without cryptographic implementation flaws (e.g., using weak but correctly implemented algorithms like MD5)
19. Clickjacking vulnerabilities due only to missing X-Frame-Options headers
20. Insecure direct object references that are purely configuration issues
21. Open ports without associated exploitable services (e.g., properly functioning services that are unnecessarily exposed)
22. Directory listing enabled on web servers without sensitive information exposure
23. Excessive error verbosity without actual exploitable vulnerabilities
24. Unencrypted data transmissions where encryption was possible but not implemented
25. Cross-site request forgery vulnerabilities due solely to missing tokens
26. Inappropriate cookie settings (e.g., missing secure or HttpOnly flags) without code-based exploitation paths
27. Unvalidated redirects that don't lead to code execution or information disclosure
28. Unnecessary service accounts with proper permissions (a configuration choice, not a code issue)
29. Server-side request forgery (SSRF) protection bypass via DNS rebinding without code flaws

"""

prompt = PromptTemplate(template=template, input_variables=["cve_details"])
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4", temperature=0.2)

# Use RunnablePassthrough to create a RunnableSequence
chain = {"cve_details": RunnablePassthrough()} | prompt | llm

def fetch_cve_details(cve_ids: List[str]) -> List[Dict]:
    """
    Fetch CVE details from the NVD API for a given list of CVE IDs using an API key.
    """
    cve_details = []

    for cve_id in cve_ids:
        retries = 0
        while retries < MAX_RETRIES:
            try:
                # Make the API request with the API key
                headers = {"apiKey": NVD_API_KEY} if NVD_API_KEY else {}
                response = requests.get(f"{NVD_API_URL}?cveId={cve_id}", headers=headers)
                response.raise_for_status()  # Raise an exception for HTTP errors

                # Parse the response
                data = response.json()
                vulnerabilities = data.get("vulnerabilities", [])

                if not vulnerabilities:
                    print(f"No data found for CVE: {cve_id}")
                    break

                # Extract relevant details
                for vuln in vulnerabilities:
                    cve_data = vuln.get("cve", {})
                    
                    # Get the description
                    descriptions = cve_data.get("descriptions", [])
                    description = next((desc["value"] for desc in descriptions if desc.get("lang") == "en"), 
                                      "No description available")
                    
                    # Get metrics information including CVSS score
                    metrics = cve_data.get("metrics", {})
                    severity_level = "Unknown"
                    cvss_score = "Unknown"
                    
                    # Try to extract CVSS metrics in order of preference
                    if "cvssMetricV31" in metrics:
                        cvss_data = metrics["cvssMetricV31"][0]["cvssData"]
                        severity_level = cvss_data.get("baseSeverity", "Unknown")
                        cvss_score = str(cvss_data.get("baseScore", "Unknown"))
                    elif "cvssMetricV30" in metrics:
                        cvss_data = metrics["cvssMetricV30"][0]["cvssData"]
                        severity_level = cvss_data.get("baseSeverity", "Unknown")
                        cvss_score = str(cvss_data.get("baseScore", "Unknown"))
                    elif "cvssMetricV2" in metrics:
                        cvss_data = metrics["cvssMetricV2"][0]["cvssData"]
                        severity_level = cvss_data.get("baseSeverity", "Unknown")
                        cvss_score = str(cvss_data.get("baseScore", "Unknown"))
                    
                    # Get affected packages and potential attack vectors
                    affected_packages = []
                    
                    configurations = cve_data.get("configurations", [])
                    for config in configurations:
                        nodes = config.get("nodes", [])
                        for node in nodes:
                            cpe_matches = node.get("cpeMatch", [])
                            for cpe_match in cpe_matches:
                                affected_packages.append(cpe_match.get("criteria", "Unknown"))
                                
                    # Get weakness information that might indicate vulnerability type
                    weakness_info = []
                    weaknesses = cve_data.get("weaknesses", [])
                    for weakness in weaknesses:
                        for desc in weakness.get("description", []):
                            weakness_info.append(desc.get("value", ""))
                    
                    cve_details.append({
                        "cve_id": cve_id,
                        "description": description,
                        "severity": {
                            "level": severity_level,
                            "score": cvss_score
                        },
                        "affected_packages": affected_packages,
                        "weakness_info": weakness_info
                    })

                break  # Exit retry loop if successful

            except requests.exceptions.RequestException as e:
                print(f"Error fetching data for CVE {cve_id}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Retrying ({retries}/{MAX_RETRIES})...")
                    time.sleep(RATE_LIMIT_WAIT_TIME)
                else:
                    print(f"Max retries reached for CVE {cve_id}. Skipping...")

        time.sleep(RATE_LIMIT_WAIT_TIME)  # Respect rate limits between requests

    return cve_details

def truncate_text(text, max_length):
    """Truncate text to a maximum length and add ellipsis if truncated."""
    if len(text) <= max_length:
        return text
    return text[:max_length] + "..."

def format_cve_details_for_llm(cve_details: List[Dict]) -> str:
    """
    Format CVE details in a token-efficient way for LLM analysis.
    """
    formatted_details = []
    
    for cve in cve_details:
        # Truncate description to control token count
        description = truncate_text(cve['description'], MAX_DESCRIPTION_LENGTH)
        
        cve_str = f"CVE ID: {cve['cve_id']}\n"
        cve_str += f"Description: {description}\n"
        cve_str += f"Severity: {cve['severity']['level']} (Score: {cve['severity']['score']})\n"
        
        # Format affected packages more efficiently
        if cve['affected_packages']:
            # Only include a limited number of packages
            packages = cve['affected_packages'][:MAX_PACKAGES_SHOWN]
            # Extract key components from CPE strings to save tokens
            simplified_packages = []
            for pkg in packages:
                # Extract product and vendor from CPE string if possible
                match = re.search(r':([^:]+):([^:]+):', pkg)
                if match:
                    vendor, product = match.groups()
                    simplified_packages.append(f"{vendor}/{product}")
                else:
                    # Use the last part of the CPE if parsing fails
                    parts = pkg.split(':')
                    simplified_packages.append(parts[-1] if len(parts) > 2 else pkg)
            
            cve_str += "Affected Components: " + ", ".join(simplified_packages)
            if len(cve['affected_packages']) > MAX_PACKAGES_SHOWN:
                cve_str += f" (+{len(cve['affected_packages']) - MAX_PACKAGES_SHOWN} more)"
            cve_str += "\n"
        
        # Include only the most relevant weakness info
        if cve['weakness_info']:
            # Just use the first weakness to save tokens
            cve_str += f"Weakness: {truncate_text(cve['weakness_info'][0], 100)}\n"
                
        formatted_details.append(cve_str)
    
    return "\n" + "-" * 20 + "\n".join(formatted_details)

def chunk_cve_ids(cve_ids, batch_size=MAX_CVE_PER_BATCH):
    """Split CVE IDs into smaller batches for processing."""
    for i in range(0, len(cve_ids), batch_size):
        yield cve_ids[i:i + batch_size]

def analyze_code_related_cves(cve_ids: List[str]) -> Dict:
    """
    Analyze CVEs in batches to manage token limits.
    """
    all_results = []
    all_cve_statuses = []  # List to store status of all CVEs for Excel
    total_analyzed = 0
    
    # Process CVEs in batches to manage token limits
    batches = list(chunk_cve_ids(cve_ids))
    print(f"Processing {len(cve_ids)} CVEs in {len(batches)} batches...")
    
    for i, batch in enumerate(batches):
        print(f"\nProcessing batch {i+1}/{len(batches)} ({len(batch)} CVEs)...")
        
        # Fetch details for this batch
        cve_details = fetch_cve_details(batch)
        total_analyzed += len(cve_details)
        
        if not cve_details:
            print("No details found for this batch")
            continue
        
        # Format the CVE details for the LLM
        formatted_details = format_cve_details_for_llm(cve_details)

        # **Debug Statement: Print Input to LLM**
        print("\n=== Input to LLM ===")
        print(formatted_details)
        print("====================\n")
        
        # Invoke the chain with the CVE details
        print(f"Analyzing batch {i+1}...")
        result = chain.invoke({"cve_details": formatted_details}).content

        # **Debug Statement: Print Raw LLM Output**
        print("\n=== Raw LLM Output ===")
        print(result)
        print("======================\n")
        
        # Try to parse the result as JSON
        try:
            # First, try to find JSON in the response
            json_match = re.search(r'\[[^\[\]]*(\[[^\[\]]*\][^\[\]]*)*\]', result)
            if json_match:
                parsed_result = json.loads(json_match.group(0))
            else:
                parsed_result = json.loads(result)
                
            # Store all CVE results for Excel
            all_cve_statuses.extend(parsed_result)
            
            # Store code-related CVEs for summary output
            code_related = [cve for cve in parsed_result if cve.get('code_related') is True]
            all_results.extend(code_related)
            
            print(f"Found {len(code_related)} code-related CVEs in this batch")
            
        except json.JSONDecodeError:
            print("Warning: Could not parse result as JSON.")
            print("Raw output (first 100 chars):", result[:100] + "...")
            # Try to extract any valid JSON from the response using a more robust approach
            try:
                # Find anything that looks like a JSON array
                matches = re.findall(r'\[\s*\{[^[]*\}\s*\]', result.replace('\n', ''))
                if matches:
                    parsed_result = json.loads(matches[0])
                    
                    # Store all CVE results for Excel
                    all_cve_statuses.extend(parsed_result)
                    
                    # Store code-related CVEs for summary output
                    code_related = [cve for cve in parsed_result if cve.get('code_related') is True]
                    all_results.extend(code_related)
                    
                    print(f"Successfully extracted {len(code_related)} CVEs using regex")
            except Exception as e:
                print(f"JSON extraction fallback also failed: {e}")
                # Add entries with error status for the current batch
                for cve_id in batch:
                    all_cve_statuses.append({
                        "cve_id": cve_id,
                        "code_related": "Error",
                        "exclusion_reason": "Failed to parse LLM response"
                    })
    
    # Write results to Excel
    write_to_excel(all_cve_statuses)
    
    return {
        "code_related_cves": all_results,
        "total_analyzed": total_analyzed,
        "total_code_related": len(all_results)
    }

def write_to_excel(cve_statuses: List[Dict]):
    """
    Write CVE analysis results to an Excel file.
    """
    # Create DataFrame
    df = pd.DataFrame(cve_statuses)
    
    # Ensure column order and handle missing columns
    columns = ["cve_id", "code_related", "exclusion_reason"]
    for col in columns:
        if col not in df.columns:
            df[col] = ""
    
    # Reorder columns
    df = df[columns]
    
    # Fill empty exclusion reasons for code-related CVEs with "-"
    df.loc[df["code_related"] == True, "exclusion_reason"] = "-"
    
    # Write to Excel
    file_name = "cve_analysis_results_known_not_affected.xlsx"
    df.to_excel(file_name, index=False, sheet_name="CVE Analysis")
    
    print(f"\nResults written to '{file_name}'")
    print(f"Total CVEs processed: {len(df)}")
    print(f"Code-related CVEs: {sum(df['code_related'] == True)}")
    print(f"Non-code-related CVEs: {sum(df['code_related'] == False)}")
    print(f"Error processing: {sum(df['code_related'] == 'Error')}")

# Example usage
cve_ids = cves

analysis_results = analyze_code_related_cves(cve_ids)

# Print results
print("\n----- ANALYSIS RESULTS -----")
print(f"Total CVEs analyzed: {analysis_results.get('total_analyzed', 0)}")

if 'total_code_related' in analysis_results:
    print(f"Code-related CVEs found: {analysis_results['total_code_related']}")
    
    # Print each code-related CVE in a readable format
    for i, cve in enumerate(analysis_results['code_related_cves'], 1):
        print(f"\n{i}. {cve['cve_id']}")
        print(f"   Vulnerability Type: {cve.get('vulnerability_type', 'Unknown')}")
        print(f"   Affected Component: {cve.get('affected_component', 'Unknown')}")
        print(f"   Requires Code Change: {cve.get('requires_code_change', True)}")
        print(f"   Severity: {cve.get('severity', {}).get('level', 'Unknown')} ({cve.get('severity', {}).get('score', 'Unknown')})") 
else:
    print("No structured results available or parsing error occurred.")
    if 'raw_result' in analysis_results:
        print("\nRaw analysis result:")
        print(analysis_results['raw_result'])

Processing 319 CVEs in 107 batches...

Processing batch 1/107 (3 CVEs)...

=== Input to LLM ===

--------------------CVE ID: CVE-2024-12085
Description: A flaw was found in rsync which could be triggered when rsync compares file checksums. This flaw allows an attacker to manipulate the checksum length (s2length) to cause a comparison between a checksu...
Severity: HIGH (Score: 7.5)
Weakness: CWE-119

CVE ID: CVE-2025-21614
Description: go-git is a highly extensible git implementation library written in pure Go. A denial of service (DoS) vulnerability was discovered in go-git versions prior to v5.13. This vulnerability allows an atta...
Severity: HIGH (Score: 7.5)
Weakness: CWE-400

CVE ID: CVE-2025-21613
Description: go-git is a highly extensible git implementation library written in pure Go. An argument injection vulnerability was discovered in go-git versions prior to v5.13. Successful exploitation of this vulne...
Severity: CRITICAL (Score: 9.8)
Weakness: CWE-88


Analyzing batch 1.